In [61]:
import json
import numpy as np
import pandas as pd
from binance.client import Client
import e #contains the api and secret keys
import math
import Coinpaprika
import datetime
import time


#boucle pour se connecter
disconnected = True
while disconnected:
    try :
        client = Client(e.api_key, e.secret_key)
        print("vous etes connecter\n")
        disconnected = False
        connected = True
    except:
        print("impossible de se connecter\nveuillez patientez\n")


#---------fin-------------
get_info_error = True
while get_info_error:
    #get all the info of my  account
    try:
        info = client.get_margin_account()

        print('info recuperer')
        get_info_error = False
    except:
        print('impossible de recuperer les infos')
balance_dataframe= pd.DataFrame(info['userAssets'])
#supprimer BNB pour qu'il ne nous derange pas
#balance_dataframe.drop(index=52,inplace=True)

#sort the portfolio by the quantity

#sort the portfolio by the quantity
balance_dataframe.sort_values('free',ascending= False, inplace= True)
#refaire les index
balance_dataframe.reset_index(inplace = True)

#supprimer un colonnes pas important
balance_dataframe.drop(columns=['index'],inplace=True)
#--------------------------------------------------------------------





vous etes connecter

info recuperer


In [62]:
balance_dataframe.head(5)

,asset,free,locked,borrowed,interest,netAsset
0,USDT,14.59031231,0,10.61303789,0,3.97727442
1,BNB,0.00796411,0,0,0.00007542,0.00788869
2,BTC,0.0000007,0,0,0,0.0000007
3,MATIC,0,0,0,0,0
4,QTUM,0,0,0,0,0


In [169]:
margin_trades = client.get_margin_trades(symbol='BTCUSDT')
#change timestamp in datetime
for margin_trade in margin_trades:
    margin_trade['time'] = datetime.datetime.fromtimestamp(margin_trade['time']/1000)

#change in dataframes
margin_trades = pd.DataFrame(margin_trades)
#delete unuseful colums
margin_trades.drop(columns=[ 'id','orderId','isMaker','isBestMatch','isIsolated'],inplace=True)

margin_trades[['price', 'qty']] = margin_trades[['price', 'qty']].astype(float)
margin_trades['at_price'] = margin_trades['qty'] * margin_trades['price']
#margin_trades.transpose(columns = ['time','symbol','price','qty','at_price'])
margin_trades['change'] = margin_trades['price'].pct_change()

#sort the margin_trades by the time
margin_trades.sort_values('time',ascending= False, inplace= True)
#refaire les index
margin_trades.reset_index(inplace = True)

#supprimer un colonnes pas important
margin_trades.drop(columns=['index'],inplace=True)


In [170]:
margin_trades

,symbol,price,qty,commission,commissionAsset,time,isBuyer,at_price,change
0,BTCUSDT,62996.54,0.00040,0.00003444,BNB,2021-04-15 09:30:48.924,False,25.198616,0.001501
1,BTCUSDT,62902.12,0.00040,0.00003453,BNB,2021-04-15 09:26:23.519,True,25.160848,0.000522
2,BTCUSDT,62869.33,0.00040,0.00003445,BNB,2021-04-15 09:23:56.441,False,25.147732,-0.003971
3,BTCUSDT,63120.00,0.00040,0.00003453,BNB,2021-04-15 09:08:06.975,True,25.248000,0.001799
4,BTCUSDT,63006.67,0.00040,0.00003447,BNB,2021-04-15 08:29:33.525,False,25.202668,-0.000976
...,...,...,...,...,...,...,...,...,...
89,BTCUSDT,64375.67,0.00038,0.00003176,BNB,2021-04-14 09:55:24.824,True,24.462755,0.000460
90,BTCUSDT,64346.04,0.00038,0.00003177,BNB,2021-04-14 09:55:09.389,False,24.451495,-0.000797
91,BTCUSDT,64397.36,0.00038,0.00003174,BNB,2021-04-14 09:53:16.772,True,24.470997,0.000528
92,BTCUSDT,64363.36,0.00038,0.00003174,BNB,2021-04-14 09:52:47.023,False,24.458077,0.000120
